In [ ]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existin

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
import json
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
chat = ChatOpenAI(model="gpt-4-1106-preview", model_kwargs={'response_format': {"type": "json_object"}})

In [ ]:
system_message = SystemMessage(
    content='''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {
        "name": string,
        "age": int,
        "description": string
    }
    ''')

user_message = HumanMessage(content='''John is 20 years old. He is a student at the University of California,
                            Berkeley. He is a very smart student.''')

In [ ]:
json_object = chat([
    system_message,
    user_message
])

<ipython-input-6-f050e367aa0d>:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  json_object = chat([


In [ ]:
json_object

AIMessage(content='{\n    "name": "John",\n    "age": 20,\n    "description": "He is a student at the University of California, Berkeley. He is a very smart student."\n}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 88, 'total_tokens': 128, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e65b9221-6919-405e-8a81-07907c6fe2f7-0', usage_metadata={'input_tokens': 88, 'output_tokens': 40, 'total_tokens': 128})

In [ ]:
try:
    json_object = json.loads(json_object.content)
    print(json_object)
except json.decoder.JSONDecodeError:
    print(json_object.content)

{'name': 'John', 'age': 20, 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}


In [10]:
json_object

{'name': 'John',
 'age': 20,
 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}

In [11]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import SimpleJsonOutputParser

In [12]:
chat = ChatOpenAI(model="gpt-4-1106-preview", model_kwargs={'response_format': {"type": "json_object"}})

In [13]:
chat_prompt = ChatPromptTemplate.from_messages(
    ("system", """I want you to extract the person name, age and a description from the following text.
    Here is the JSON schema:
    "name": string
    "age": int
    "description": string
    {message_to_extract}
    """)
)

In [14]:
chain = (
    chat_prompt
    | chat
    | SimpleJsonOutputParser() # The output parser is optional, but it makes the output easier to parse, rather than using json.loads
)

In [15]:
result = chain.invoke({
    "message_to_extract": "I am 30 years old and my name is John Doe. I am a software engineer."
})

In [16]:
print(result)

{'name': 'John Doe', 'age': 30, 'description': 'I am a software engineer.'}
